# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [1]:
# imports
import os
import requests
import json
import ollama
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constantes
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()

MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

La clave de API parece buena


In [9]:
# Configura el entorno
link_system_prompt = "Se te proporcionará una pregunta relacionada al software. El código está \
escrito en python, quiero formules una respuesta como Markdown y que sea atractiva para leer. \
En el caso de que muestres código, hazlo de una manera legible, como si estuviera siendo presentado en un IDE."

In [12]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código, por qué, transforma el código a C++, compara la performance y comenta quien se \
comporta mejor ante un set de 1.000.000 de datos:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [13]:
# Haz que gpt-4o-mini responda, con streaming
def stream_response_gpt(system_prompt, question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [14]:
print("Respuesta de GPT\n")
stream_response_gpt(link_system_prompt, question)

Respuesta de GPT



## Análisis del Código en Python

El código proporcionado utiliza una expresión generadora para extraer de manera eficiente autores de una lista de libros. Vamos a desglosar su funcionamiento:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explicación:

1. **Conjunto de Autores**: `book.get("author") for book in books` crea un conjunto (`set`) de autores. Utiliza `book.get("author")` para acceder al valor de la clave `"author"` en cada diccionario `book`.

2. **Filtrado de Nulos**: La condición `if book.get("author")` garantiza que sólo se añadan al conjunto aquellos autores que no sean `None` o cadenas vacías.

3. **Yield**: `yield from` se utiliza aquí para generar los elementos del conjunto en lugar de devolver el conjunto completo. Esto hace que la función sea un generador, lo que permite que los resultados se procese uno a uno sin cargar todo el conjunto en memoria.

### Transformación a C++

Ahora, veamos cómo se podría implementar esto en C++:

cpp
#include <iostream>
#include <set>
#include <vector>
#include <string>
#include <optional>

struct Book {
    std::optional<std::string> author;
};

std::set<std::string> getAuthors(const std::vector<Book>& books) {
    std::set<std::string> authors;
    for (const auto& book : books) {
        if (book.author) {
            authors.insert(*book.author);
        }
    }
    return authors;
}

int main() {
    std::vector<Book> books = { /* Aquí sería donde se llenan los libros */ };
    auto authors = getAuthors(books);
    for (const auto& author : authors) {
        std::cout << author << std::endl;
    }
    return 0;
}


### Comparación de Performance

Al comparar ambas implementaciones, hay algunos factores a considerar:

1. **Uso de Memoria**:
   - **Python**: La implementación en Python usa `yield from`, lo que significa que no se necesita almacenar todos los autores en memoria al mismo tiempo, solo devuelve cada autor a medida que se necesita.
   - **C++**: La versión en C++ utiliza un conjunto (`std::set`) que almacena todos los autores y tiene que descargar todo el conjunto al final, lo que puede aumentar el uso de memoria.

2. **Velocidad de Ejecución**:
   - **Python**: Puede ser más lento en términos de tiempo de ejecución debido a la naturaleza interpretada de Python y la sobrecarga de los métodos utilizados.
   - **C++**: C++ es un lenguaje compilado y, generalmente, se ejecutará más rápido en comparación con Python, especialmente para un tamaño considerable de datos como 1.000.000 de entradas.

### Conclusión

- Si el uso de memoria es una preocupación y se requiere la generación perezosa de resultados, la versión en **Python** es más adecuada.
- En situaciones donde la **velocidad** de ejecución es crucial, especialmente con conjuntos de datos grandes, la implementación en **C++** superará a la de **Python** en rendimiento general.

Ambas versiones tienen sus ventajas dependiendo del contexto en que se utilicen.

In [10]:
# Haz que Llama 3.2 responda
def response_llama(system_prompt, question):
    response = ollama.chat(
        model=MODEL_LLAMA, 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ]
    )
    display(Markdown(response['message']['content']))

In [11]:
print("Respuesta de Llama\n")
response_llama(link_system_prompt, question)

Respuesta de Llama



**Análisis del Código Original**

El código original es escrito en Python y utiliza un recurso llamado "generador" (yield) para obtener una colección de valores. A continuación, se explica qué hace este código:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Este código utiliza varias estructuras lingüísticas simultáneas:

* `yield`: es un statement que pausa la ejecución del generador y devuelve un valor. En este caso, se utiliza para generar una secuencia de valores.
* `from`: es una palabra clave que indica el uso de un recurso llamado "generador".
* `{}`: se refiere a un set o una colección sin orden.
* `for book in books if book.get("author")`: es una estructura de control que itera sobre cada elemento en la colección `books`. La condición `if book.get("author")` filtra los elementos para que solo incluyan aquellos que tengan el valor "author" asociado.

En resumen, este código utiliza un generador para obtener una lista de autores a partir de una colección de libros. El resultado es una secuencia de valores donde cada valor corresponde a un autor en la colección original.

**Transformación al C++**

Para convertir este código al C++, podemos utilizar iteradores y mapas para lograr el mismo resultado:

```cpp
#include <iostream>
#include <vector>
#include <string>
#include <unordered_map>

// Definir una estructura para un libro
struct Libro {
    std::string autor;
};

int main() {
    // Crear una colección de libros
    std::vector<Libro> books = {
        {"J.K. Rowling", "Harry Potter"},
        {"George R.R. Martin", "A Song of Ice and Fire"},
        {"John Grisham", "The Firm"}
    };

    // Crear un mapa con autores y sus respectivos libros
    std::unordered_map<std::string, std::vector<Libro>> autoresLibros;
    for (const auto& libro : books) {
        autoresLibros[libro.autor].push_back(libro);
    }

    // Obtener la lista de autores
    for (const auto& autor : autoresLibros) {
        std::cout << autor.first << std::endl;
    }

    return 0;
}
```

En este ejemplo, creamos una estructura `Libro` para representar cada libro con un autor asociado. Luego, creamos una colección de libros y un mapa con autores como clave y sus respectivos libros en el valor.

Finalmente, iteramos sobre el mapa y obtenemos la lista de autores imprimiéndolos por pantalla.

**Comparación de Performance**

La performance del código original Python y el C++ es similar, ya que ambos utilizan estructuras lingüísticas para lograr el mismo resultado. Sin embargo, hay algunas diferencias clave:

* **Ejecución**: El código Python utiliza un motor de ejecución interno (CPython), mientras que el código C++ utiliza la compilación estática y la ejecución directa.
* **Almacenamiento**: El código C++ utiliza estructuras de datos como vectores y mapas, que pueden ser más eficientes en términos de almacenamiento y acceso rápido.
* **Optimización**: El código C++ puede ser optimizado mediante técnicas como el uso de temporizadores y reducción del árbol de recursividad.

En general, si se necesita una aplicación con rendimiento óptimo, es recomendable escribir el código en C++. Sin embargo, si se requiere una solución rápida y fácil de entender, Python puede ser una opción más adecuada.